In [ ]:
# Función de coste binaria con valores {-1, 1}
# g(z) = "\
#     +11 * z_0 - 17.5 * z_1 - 28 * z_2 - 17 * z_3 + 11.5 * z_4 \
#     +13.5 * (-z_0*z_2 + z_1*z_2 + z_2*z_3 - z_2*z_4 + z_0*z_1 - z_0*z_3 - z_1*z_4) \
#     +(6.75*z_3**2 + 6.75*z_4**2 + 13.5*(z_0**2 + z_1**2 + z_2**28)) \
#     +26.5"

# Grafo
edges = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
num_nodes = 4

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(-Z0*Z2 + Z1*Z2 + Z2*Z3 - Z2*Z4 + Z0*Z1 - Z0*Z3 - Z1*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
linear_coefs = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
quadra_coefs = {(2, 4): -13.5, (2, 3): 13.5, (1, 4): -13.5, (1, 2): 13.5,
                (0, 3): -13.5, (0, 2): -13.5, (0, 1): 13.5}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Imprime un grafo dirigido pesado
# Entrada
#     graph_nodes: Lista de nodos del grafo
#     graph_edges: Diccionario {vértice: peso} donde vértice es (n1, n2)
#     layout:      Para especificar la posición de los nodos
def print_graph(graph_nodes, graph_edges, layout=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in graph_edges.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 2))
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    #plt.savefig("../../latex/img/primer_grafo/primer_grafo.png")

# edges = {(0, 1): "5(q_0)", (0, 2): "8(q_1)", (1, 2): "2(q_2)", (1, 3): "7(q_3)", (2, 3): "4(q_4)"}
print_graph(range(num_nodes), edges, layout={0: [-1, 0], 1: [0, 1],
                                             2: [0, -1], 3: [1, 0]})

In [ ]:
# Función de coste binaria con valores {0, 1}
cost_function = "\
5*x_01 + 8*x_02 + 2*x_12 + 7*x_13 + 4*x_23 + \
27*(x_01 + x_02 - 1)**2 + \
27*(x_01 - x_12 - x_13)**2 + \
27*(x_02 + x_12 - x_23)**2"

# Evalúa una solución concreta según la función de coste para la versión QUBO
# Entrada:
#     - bits: Cadena con los valores de los qubits medidos
#             Orden: q_4 q_3 q_2 q_1 q_0
def eval_cost_function(bits):
    assert len(edges) == len(bits), "Error in cost_function: Length of bits"

    inv_bits = bits[::-1]  # Los qubits están en orden inverso

    param_dict = {}
    # Asignación entre x_ij -> q_n
    # Establecido por el orden de edges. Ej: x_12 -> q_2 (esto es, qubit nº2)
    for n, (i, j) in enumerate(edges):
        param_dict[f"x_{i}{j}"] = int(inv_bits[n])

    return eval(cost_function, param_dict)

In [ ]:
from qiskit import QuantumCircuit

def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = len(edges)  # Tantos qubits como aristas tenga el grafo
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idx, coef in enumerate(linear_coefs):
            circuit.rz(coef, q_idx)
        for q_idxs, coef in quadra_coefs.items():
            circuit.rzz(coef * gamma[p], q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    circuit.measure_all()
    return circuit

In [ ]:
def compute_expectation(counts):
    media = 0
    len_count = 0
    for bits, count in counts.items():
        cost = eval_cost_function(bits)
        media += cost * count
        len_count += count

    return media/len_count

In [ ]:
# Simulador
from qiskit import Aer
from scipy.optimize import minimize

backend = Aer.get_backend('aer_simulator')
shots = 1024

def execute_circuit(theta):
    qc = generate_qaoa_circuit(theta)
    counts = backend.run(qc, shots=shots).result().get_counts()
    return compute_expectation(counts)

num_layers = 1
theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
theta_res

In [ ]:
from qiskit.visualization import plot_histogram
backend = Aer.get_backend('aer_simulator')
shots = 1024

qc = generate_qaoa_circuit(theta_res.x)
counts = backend.run(qc, shots=shots).result().get_counts()
#plot_histogram(normalized_counts, figsize=(13, 6.83),
#               filename="../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_aer_resultado.png")
normalized_counts = {key: val / shots for (key, val) in counts.items()}
plot_histogram(normalized_counts, figsize=(13, 6.83))

In [ ]:
# Gamma function
import numpy as np
import matplotlib.pyplot as plt

def gamma_function():
    x = np.linspace(0.3, 1.5, 120)
    y = []
    beta = 1.0
    for gamma in x:
        y.append(execute_circuit([beta, gamma]))

    plt.plot(x, y)
    # plt.savefig("../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_p_27_gamma_fun.png")
    plt.show()

gamma_function()

In [ ]:
# 3D Gamma function
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools

def gamma_function_3d():
    beta = np.arange(0.3, 1.5, 0.01)
    gamma = np.arange(0.3, 1.5, 0.01)
    beta, gamma = np.meshgrid(beta, gamma)
    z = np.zeros([len(beta), len(beta)])

    for i in range(len(beta)):
        for j in range(len(beta)):
            z[i][j] = execute_circuit([beta[i][j], gamma[i][j]])

    fig = plt.figure()
    axis = plt.axes(projection='3d')
    figure = axis.plot_surface(gamma, beta, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    fig.colorbar(figure, shrink=0.5, aspect=5)
    # plt.savefig("../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_p_27_gamma_fun.png")
    plt.show()

gamma_function_3d()

In [ ]:
# Statistics
def max_global_statistics(num_layers=1, num_generations=1000):
    max_statistics = {}
    global_statistics = {}
    for iteration in range(0, num_generations):
        theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
        qc = generate_qaoa_circuit(theta_res.x)
        counts = backend.run(qc, shots=shots).result().get_counts()

        # Max statistics
        max_path = max(counts, key=counts.get)
        if max_path not in max_statistics:
            max_statistics[max_path] = 0
        max_statistics[max_path] += 1

        # Global statistics
        for (path, num_appearances) in counts.items():
            if path not in global_statistics:
                global_statistics[path] = 0
            global_statistics[path] += num_appearances

    for path in global_statistics:
        global_statistics[path] = global_statistics[path] / shots / num_generations  # Normalize

    for path in max_statistics:
        max_statistics[path] = max_statistics[path] / num_generations  # Normalize

    max_statistics = sorted(max_statistics.items(), key=lambda x: x[1], reverse=True)
    global_statistics = sorted(global_statistics.items(), key=lambda x: x[1], reverse=True)
    return max_statistics, global_statistics

max_num_layers = 3
for p in range(1, max_num_layers + 1):
    max_st, global_st = max_global_statistics(num_layers=p, num_generations=1000)
    print(f"Max statistics ({p}): ", max_st)
    print(f"Global statistics({p}): ", global_st)

In [ ]:
# Circuito con Rzz ordenadas como el paper
# Max statistics (1):  [('10101', 0.908), ('10110', 0.092)]
# Global statistics(1):  [('10101', 0.390283203125), ('10001', 0.1988779296875), ('00101', 0.0467822265625), ('10110', 0.0446611328125), ('11001', 0.0326279296875), ('00001', 0.0319658203125), ('10010', 0.0307470703125), ('11101', 0.030072265625), ('10100', 0.02943359375), ('10111', 0.0289248046875), ('10000', 0.026359375), ('10011', 0.0230107421875), ('00111', 0.0127265625), ('00011', 0.009048828125), ('00000', 0.0066376953125), ('01001', 0.0064013671875), ('00010', 0.0059541015625), ('11011', 0.0058623046875), ('00100', 0.00495703125), ('00110', 0.0047392578125), ('11100', 0.004658203125), ('11000', 0.0044775390625), ('11111', 0.0042626953125), ('01101', 0.0040849609375), ('11010', 0.00405859375), ('01011', 0.0024521484375), ('01111', 0.001556640625), ('01000', 0.001302734375), ('11110', 0.0011279296875), ('01100', 0.000908203125), ('01010', 0.000693359375), ('01110', 0.00034375)]
# Max statistics (2):  [('10101', 0.667), ('10010', 0.079), ('10110', 0.052), ('10001', 0.047), ('11010', 0.041), ('11011', 0.03), ('11001', 0.028), ('01001', 0.021), ('11101', 0.009), ('10011', 0.006), ('00101', 0.005), ('00001', 0.003), ('01101', 0.003), ('11000', 0.003), ('10000', 0.002), ('00000', 0.001), ('01010', 0.001), ('00010', 0.001), ('10100', 0.001)]
# Global statistics(2):  [('10101', 0.2393779296875), ('11101', 0.07155859375), ('10010', 0.064576171875), ('10110', 0.0462412109375), ('10111', 0.0459267578125), ('10001', 0.042607421875), ('11001', 0.0419658203125), ('10100', 0.0416884765625), ('00101', 0.0412744140625), ('11011', 0.0362119140625), ('10000', 0.0356962890625), ('10011', 0.0343154296875), ('00001', 0.0297119140625), ('01001', 0.029453125), ('01101', 0.0267236328125), ('11010', 0.025740234375), ('00000', 0.0246826171875), ('11000', 0.0160556640625), ('11111', 0.0132734375), ('00010', 0.0105048828125), ('00100', 0.0100625), ('00011', 0.009771484375), ('11110', 0.008208984375), ('01000', 0.0080322265625), ('11100', 0.00757421875), ('01010', 0.007462890625), ('00110', 0.006279296875), ('00111', 0.0060634765625), ('01011', 0.0060087890625), ('01111', 0.0045126953125), ('01110', 0.0045078125), ('01100', 0.0039296875)]
# Max statistics (3):  [('10101', 0.595), ('10001', 0.103), ('10010', 0.079), ('01001', 0.065), ('00000', 0.048), ('11011', 0.024), ('11101', 0.023), ('01101', 0.014), ('10100', 0.01), ('00001', 0.007), ('00101', 0.006), ('10110', 0.005), ('11010', 0.004), ('11001', 0.004), ('00010', 0.003), ('10000', 0.002), ('10011', 0.002), ('10111', 0.002), ('01011', 0.002), ('01010', 0.001), ('11000', 0.001)]
# Global statistics(3):  [('10101', 0.1958486328125), ('10001', 0.0810615234375), ('00000', 0.057326171875), ('11011', 0.0501123046875), ('10010', 0.0478271484375), ('00101', 0.0410537109375), ('11101', 0.039595703125), ('01010', 0.03657421875), ('10111', 0.034267578125), ('00001', 0.032943359375), ('01001', 0.0327353515625), ('10100', 0.0320908203125), ('01101', 0.0308662109375), ('00010', 0.027072265625), ('11010', 0.0254892578125), ('10110', 0.02493359375), ('11001', 0.0238369140625), ('01011', 0.022009765625), ('00111', 0.02046875), ('11000', 0.019390625), ('11111', 0.01890234375), ('01000', 0.015828125), ('10011', 0.0150244140625), ('00100', 0.0133515625), ('10000', 0.0119736328125), ('11100', 0.010228515625), ('00011', 0.0084619140625), ('01111', 0.007455078125), ('01110', 0.0064775390625), ('00110', 0.006458984375), ('11110', 0.00578125), ('01100', 0.004552734375)]

In [ ]:
!dunstify -t $((30* 1000*60)) --urgency=critical "Ejecución completada"